# Receipt printing in .NET (ESC/POS)

Second way to print receipts is to prepare a text document and print it as plain text. This is more portable and works on all platforms, including Linux and Mac OS. The code below prepares a simple receipt in a text format and sends it to the printer.

## What are ESC/POS commands?

ESC/POS (_Epson Standard Code for Point Of Sale_) is a de‑facto standard command set used by most receipt printers (Epson and many compatibles). Instead of sending raster data, your app streams plain text mixed with small binary control sequences that change printer behavior: font, alignment, barcode/QR, images, paper cutting, and opening the cash drawer.

### How it works at a glance

1. You open the printer as a stream (e.g., a COM port).
2. You write text bytes encoded in the printer’s expected code page.
3. You interleave control sequences to format text, print barcodes, QR codes or cut paper.
4. Many printers buffer until a line feed (`LF`/`0A`) or until the buffer is flushed.

### Limitations and gotchas

* Code pages: Make sure characters you need exist in the active code page.
* Paper width: usually 58 mm or 80 mm impacts max columns.
* Spooling: On desktops, printing through a driver can translate data; for raw ESC/POS, use a raw/direct send method.
* Vendor quirks: Cash drawer pins, image density, and QR models vary; consult the manual.

### Typical ESC/POS Commands

Command                 | Characters      | Hex Sequence     | Comments
----------------------- | --------------- | ---------------- | --------
Initialize printer      | `ESC @`         | `1B 40`          |
New line                | `LF`            | `0A`             |
Bold on                 | `ESC E 1`       | `1B 45 01`       |
Bold off                | `ESC E 0`       | `1B 45 00`       |
Underline on            | `ESC - 1`       | `1B 2D 01`       |
Underline off           | `ESC - 0`       | `1B 2D 00`       |
Double width on         | `GS ! 32`       | `1D 21 20`       |  
Double width off        | `GS ! 0`        | `1D 21 00`       |  
Double height on        | `GS ! 16`       | `1D 21 10`       |  
Double height off       | `GS ! 0`        | `1D 21 00`       |  
Align left              | `ESC a 0`       | `1B 61 00`       |
Align center            | `ESC a 1`       | `1B 61 01`       |
Align right             | `ESC a 2`       | `1B 61 02`       |
Cut (full)              | `GS V 0`        | `1D 56 00`       | Model-dependent
Cut (partial)           | `GS V 1`        | `1D 56 01`       | Model-dependent
Open cash drawer (kick) | `ESC p m t1 t2` | `1B 70 00 19 FA` | Model-dependent, `t1` and `t2` are timings

## Using ESC/POS printers from .NET

First, we have to be able to communicate with the printer directly. If it's serial or direct network printer, it's easy: just open stream using `SerialPort` or `NetworkStream`. Most printers today are connected over USB and available as standard graphical printer in Windows.

### Find available printers

We will need Windows printer names for the `RawPrint.NetStd` library. Instead of using the `System.Drawing.Common` package (which is fairly heavyweight), we can directly query the registry:

In [95]:
var printers = Microsoft.Win32.Registry.LocalMachine.OpenSubKey(@"SYSTEM\CurrentControlSet\Control\Print\Printers", false);
var printerNames = printers.GetSubKeyNames();

foreach (var name in printerNames) {
    Console.WriteLine(name);
}

Adobe PDF
Canon MF645C
Microsoft Print to PDF
OneNote (Desktop)
XP-58


### Using RawPrint.NetStd

Using the `RawPrint.NetStd` library we can send them raw commands, however. Let's reference the package:

In [96]:
#r "nuget: RawPrint.NetStd"

Installed Packages RawPrint.NetStd, 1.0.0

Now we can prepare the command stream and send it directly to the printer:

In [97]:
using System.IO;

using (var s = new MemoryStream())
using (var w = new StreamWriter(s)) {
    // Prepare ESC/POS commands in a memory stream
    w.Write("\x1B\x40");            // Initialize printer
    w.WriteLine("Hello, World!");   // Print text
    w.Write("\x1B\x45\x01");        // Bold ON
    w.WriteLine("Bold text");       // Print bold text
    w.Write("\x1B\x45\x00");        // Bold OFF
    w.WriteLine();
    w.WriteLine();
    w.WriteLine();
    w.WriteLine();
    w.Flush();

    // Send to printer
    var prn = new RawPrint.NetStd.Printer();
    prn.PrintRawStream("XP-58", s, documentName: "Test document", paused: false);
}

### Using ESCPOS

Creating the ESC/POS commands manually and sending them in low level way it not a great fun, though. We can use the `ESCPOS` library to generate the stream for us instead.

Let's reference the package first:

In [98]:
#r "nuget: ESCPOS"

using ESCPOS;
using ESCPOS.Utils;

Installed Packages ESCPOS, 1.3.0

Now we can format the document using properties and methods of `Commands` class:

In [99]:
using (var s = new MemoryStream()) {
    // Prepare ESC/POS commands in a memory stream
    s.Write(Commands.InitializePrinter);    // Initialize printer
    s.Write("Hello, World!".ToBytes());     // Print text
    s.Write(Commands.LineFeed);
    s.Write(Commands.DoubleStrikeOn);       // Bold ON
    s.Write("Bold text".ToBytes());         // Print bold text
    s.Write(Commands.DoubleStrikeOff);      // Bold OFF
    s.Write(Commands.LineFeed);
    s.Write(Commands.LineFeed);
    s.Write(Commands.LineFeed);
    s.Write(Commands.LineFeed);

    // Send to printer
    var prn = new RawPrint.NetStd.Printer();
    prn.PrintRawStream("XP-58", s, documentName: "Test document", paused: false);
}

## Create receipt using ESC/POS mode

To make working with ESCPOS library easier, I created helper class `EscPosDocument`. It creates a memory stream with the data, which can be then printed later.

It also contains some utility methods for writing receipts.

There is notorious problem with characters with diacritics, as not every printer supports them. To overcome this problem in the most universal way, I just strip all diacritic marks from letters, because I'm lazy :)

In [100]:
public class EscPosDocument {

    // Constructor

    public EscPosDocument() { 
        this.Write(Commands.InitializePrinter);
    }

    // Properties

    public Stream Stream { get; } = new MemoryStream();

    public int Columns { get; set; } = 32; // Typical values are 32 for 58 mm paper and 48 for 80 mm paper

    public float ReceiptTotal { get; private set; } = 0;

    public Func<string, byte[]> StringToBytes { get; init; } = RemoveDiacritic;

    // Basic write methods

    public void Write(byte[] bytes) => this.Stream.Write(bytes, 0, bytes.Length);

    public void Write(string text) => this.Write(this.StringToBytes(text));

    public void Write(params object[] items) {
        foreach (var item in items) {
            if (item is string s) {
                this.Write(s);
            } else if (item is byte[] b) {
                this.Write(b);
            } else {
                this.Write(item.ToString());
            }
        }
    }

    // Utility methods for writing lines

    public void WriteLine() => this.Write(Commands.LineFeed);

    public void WriteLine(string text) => this.Write(text, Commands.LineFeed);

    public void WriteLine(params object[] items) => this.Write(items.Append(Commands.LineFeed).ToArray());

    public void WriteDoubleAlignedLine(string left, string right) {
        var totalLength = left.Length + 1 + right.Length;
        if (totalLength > this.Columns) {
            // Too long, just write left and right on separate lines
            this.WriteLine(Commands.AlignToLeft, left);
            this.WriteLine(Commands.AlignToRight, right);
            this.Write(Commands.AlignToLeft);
        } else {
            // It will fit, so align left and right on the same line
            var spaces = new string(' ', this.Columns - left.Length - right.Length);
            this.WriteLine(Commands.AlignToLeft, left, spaces, right);
        }
    }

    public void WriteLines(int numberOfLines) {
        for (int i = 0; i < numberOfLines; i++) this.WriteLine();
    }

    // Other utility methods

    public void WriteHorizontalLine(char ch = '-') => this.WriteLine(new string(ch, this.Columns));

    //  Receipt specific methods

    public void WriteReceiptItem(string name, float unitPrice, int quantity) {
        var total = unitPrice * quantity;
        var priceInfo = quantity == 1 ? $"{unitPrice:N2} Kč" : $"{quantity} x {unitPrice:N2} Kč = {total:N2} Kč";
        this.ReceiptTotal += total;
        this.WriteDoubleAlignedLine(name, priceInfo);
    }

    public void WriteReceiptTotal() {
        this.Write(Commands.DoubleStrikeOn);
        this.WriteDoubleAlignedLine("Celkem", $"{this.ReceiptTotal:N2} Kč");
        this.Write(Commands.DoubleStrikeOff);
    }

    // Helper method to convert string to byte array and remove diacritics
    
    public static byte[] RemoveDiacritic(string text) {
        var normalized = text.Normalize(NormalizationForm.FormD);
        var sb = new System.Text.StringBuilder();
        foreach (var c in normalized) {
            var uc = System.Globalization.CharUnicodeInfo.GetUnicodeCategory(c);
            if (uc != System.Globalization.UnicodeCategory.NonSpacingMark) sb.Append(c);
        }
        var ascii = sb.ToString().Normalize(NormalizationForm.FormC);
        return System.Text.Encoding.ASCII.GetBytes(ascii);
    }

}

Now we can use the helper class to print a sample receipt:

In [101]:
var doc = new EscPosDocument();

// Write header
doc.WriteLine(Commands.AlignToCenter, Commands.CharSizeDoubleHeight, "Altairis, s. r. o.", Commands.CharSizeReset);
doc.WriteLine("Bořivojova 35, 130 00 Praha 3");
doc.WriteLine("IČO: 27560911, DIČ: CZ27560911");
doc.WriteHorizontalLine('_');
doc.WriteLine();

// Write receipt items and total
doc.WriteReceiptItem("Položka 1", 49.00f, 1);
doc.WriteReceiptItem("Nějaký opravdu velmi dlouhý název položky 2", 199.00f, 2);
doc.WriteReceiptItem("Položka 3", 9.90f, 1);
doc.WriteReceiptItem("Položka 4", 25.00f, 5);
doc.WriteReceiptItem("Položka 5", 29.90f, 1);
doc.WriteHorizontalLine();
doc.WriteReceiptTotal();
doc.WriteLine();

// Write receipt number as a barcode
doc.Write(Commands.AlignToCenter, Commands.ToBarcode("123456789", BarCodeType.ITF, heightInDots: 60));
doc.WriteLine("123456789");
doc.WriteLine();

// Write footer
doc.WriteLine(DateTime.Now.ToString("dd. MM. yyyy HH:mm"));
doc.Write(Commands.AlignToLeft);

// Add some blank lines at the end to eject the paper
doc.WriteLines(5);

// Send to printer
var prn = new RawPrint.NetStd.Printer();
prn.PrintRawStream("XP-58", doc.Stream, documentName: "Test document", paused: false);